In [1]:
import pandas as pd
import json
import requests

# Tratamento dos dados

## Importando Bases de Dados

In [2]:
goals = pd.read_csv('goals.csv')
matches = pd.read_csv('matches.csv', dtype={"match_type": "category"})

## Informações Sobre os Dados - Matches

In [3]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   match_id        100000 non-null  object  
 1   match_type      100000 non-null  category
 2   player_1        100000 non-null  object  
 3   player_2        100000 non-null  object  
 4   match_date      100000 non-null  object  
 5   match_duration  100000 non-null  object  
dtypes: category(1), object(5)
memory usage: 3.9+ MB


In [4]:
matches.sample(10)

,match_id,match_type,player_1,player_2,match_date,match_duration
27341,M02136636,Casual,VF2F5176,MC4B2517,2020-03-23,00:10:00
31843,M02141138,Ranked,MS9I7465,LA6F5194,2020-01-20,00:10:00
4687,M02113982,Ranked,CF9T7117,NV8G8511,2020-05-01,00:10:00
82000,M02191295,Casual,VJ6B1346,CY4O6595,2020-02-28,00:10:00
46697,M02155992,Casual,ZP2S8445,CW6Z7132,2019-11-10,00:10:00
72811,M02182106,Ranked,BW2D9149,HB1E7183,2020-04-18,00:10:00
41693,M02150988,Ranked,NW0H7923,YH4P8441,2020-09-09,00:10:00
11093,M02120388,Ranked,IE2F5413,RN2Z9493,2019-09-01,00:10:00
73408,M02182703,Casual,BD5P6965,HW9R5262,2019-08-25,00:10:00
50009,M02159304,Ranked,HT9R6714,WD7B3216,2019-10-09,00:10:00


In [5]:
matches.describe()

,match_id,match_type,player_1,player_2,match_date,match_duration
count,100000,100000,100000,100000,100000,100000
unique,100000,2,10000,9998,731,2
top,M02189151,Casual,VR2K3833,HX3D6186,2019-03-01,00:10:00
freq,1,50302,24,25,300,94050


In [6]:
goals.head()

,goal_id,match_id,goal_timestamp,player_id
0,1,M02178449,00:09:48,SV4W8695
1,2,M02201817,00:01:56,WT9P1527
2,3,M02148361,00:03:48,OI7P6261
3,4,M02143962,00:06:16,EA3H6562
4,5,M02152784,00:04:38,UE9C4827


## Número de gols por jogador

Identificar em cada Partida:
* Quem venceu e quem perdeu o jogo;
* Quantos gols foram marcados por cada jogador;
* Se a vitória foi fácil;

### Inserindo colunas ['goals_players_1', 'goals_players_2', 'winner', 'defeated', 'easy_win'] no dataframe matches

In [ ]:
for index, row in matches.iterrows():
  goals_player_1 = goals.query("match_id == @row.match_id and player_id == @row.player_1")['goal_id'].count()
  goals_player_2 = goals.query("match_id == @row.match_id and player_id == @row.player_2")['goal_id'].count()
  winner = row.player_1 if goals_player_1 > goals_player_2 else row['player_2']
  defeated = row.player_2 if goals_player_1 > goals_player_2 else row['player_1']
  easy_win = abs(goals_player_1 - goals_player_2) > 4
  matches.at[index, 'goals_player_1'] = goals_player_1
  matches.at[index, 'goals_player_2'] = goals_player_2
  matches.at[index, 'winner'] = winner
  matches.at[index, 'defeated'] = defeated
  matches.at[index, 'easy_win'] = easy_win

In [0]:
# matches.to_csv('matches_with_goals.csv')

## Importando base 'matches_with_goals.csv'


In [7]:
matches = pd.read_csv('matches_with_goals.csv')

In [8]:
matches.head()

,match_id,match_type,player_1,player_2,match_date,match_duration,goals_player_1,goals_player_2,winner,defeated,easy_win
0,M02109295,Ranked,DA1X9864,IA8T7256,2019-05-20,00:10:00,5.0,0.0,DA1X9864,IA8T7256,True
1,M02109296,Casual,SZ0S2637,CK8P4729,2020-11-18,00:10:00,5.0,0.0,SZ0S2637,CK8P4729,True
2,M02109297,Ranked,QX0H6977,PK9E7375,2020-02-28,00:10:00,2.0,0.0,QX0H6977,PK9E7375,False
3,M02109298,Casual,BK3W3327,QZ7C5416,2019-07-01,00:10:00,3.0,0.0,BK3W3327,QZ7C5416,False
4,M02109299,Casual,EG5I9716,GN4N5747,2020-12-11,00:10:00,6.0,0.0,EG5I9716,GN4N5747,True


## Identificando Todos os Usuários e criando novo DataFrame com as estatísticas de cada um

In [17]:
players = pd.DataFrame(pd.unique(matches[['player_1', 'player_2']].values.ravel('K')))

In [18]:
players.columns = ['player_id']
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   player_id  10000 non-null  object
dtypes: object(1)
memory usage: 78.2+ KB


### Contando número de empates

In [19]:
matches[matches['goals_player_1'] == matches['goals_player_2']]

,match_id,match_type,player_1,player_2,match_date,match_duration,goals_player_1,goals_player_2,winner,defeated,easy_win,tied
11,M02109306,Casual,DI5Z8931,ER0B3239,2020-12-20,00:10:00,0.0,0.0,-,-,False,True
272,M02109567,Casual,CK4J3439,IU0C4745,2019-03-29,00:10:00,0.0,0.0,-,-,False,True
310,M02109605,Casual,QQ2S6557,QQ2S6557,2019-07-08,00:10:00,5.0,5.0,-,-,False,True
331,M02109626,Casual,UU7F4156,TU0X6811,2019-09-23,00:10:00,0.0,0.0,-,-,False,True
532,M02109827,Casual,YY7P8486,DZ8P8886,2019-03-02,00:10:00,0.0,0.0,-,-,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
99324,M02208619,Casual,QM4S7539,WL2I9348,2019-04-19,00:10:00,0.0,0.0,-,-,False,True
99503,M02208798,Casual,QI0L6968,CS5G5255,2019-07-11,00:15:00,0.0,0.0,-,-,False,True
99557,M02208852,Ranked,MO0O7817,DD0N2982,2019-12-14,00:10:00,0.0,0.0,-,-,False,True
99563,M02208858,Casual,BK6L3571,UJ9C6129,2019-06-01,00:10:00,0.0,0.0,-,-,False,True


In [20]:
matches['tied'] = matches['goals_player_1'] == matches['goals_player_2']

In [21]:
for index, row in matches.iterrows():
    if row['tied'] is True:
        matches.at[index, 'winner'] = '-'
        matches.at[index, 'defeated'] = '-'

## Identificar em cada Jogador:
* Número de jogos;
* Número de vitórias;
* Número de derrotas;
* Percentual de aproveitamento;
* Número de goleadas aplicadas;
* Número de goleadas sofridas;


In [22]:
for index, row in players.iterrows():
    matches_count = matches.query("player_1 == @row.player_id or player_2 == @row.player_id")['match_id'].count()
    wins_count = matches.query("winner == @row.player_id")['match_id'].count()
    easy_wins_count = matches.query("winner == @row.player_id and easy_win == True")['match_id'].count()
    defeated_count = matches.query("defeated == @row.player_id")['match_id'].count()
    easy_defeated_count = matches.query("defeated == @row.player_id and easy_win == True")['match_id'].count()
  
    players.at[index, 'matches_count'] = matches_count
    players.at[index, 'wins_count'] = wins_count
    players.at[index, 'easy_wins_count'] = easy_wins_count
    players.at[index, 'defeated_count'] = defeated_count
    players.at[index, 'easy_defeated_count'] = easy_defeated_count
    players.at[index, 'tied'] = matches_count - wins_count - defeated_count

In [28]:
players['score'] = ((players['wins_count'] * 3) + players['tied']) / (players['matches_count'] * 3)

In [29]:
players.head()

,player_id,matches_count,wins_count,easy_wins_count,defeated_count,easy_defeated_count,tied,score
0,DA1X9864,24.0,9.0,4.0,15.0,9.0,0.0,0.375000
1,SZ0S2637,22.0,14.0,10.0,8.0,6.0,0.0,0.636364
2,QX0H6977,20.0,11.0,7.0,8.0,7.0,1.0,0.566667
3,BK3W3327,14.0,11.0,6.0,3.0,3.0,0.0,0.785714
4,EG5I9716,22.0,12.0,9.0,10.0,7.0,0.0,0.545455


## Verificando correlação das variáveis

In [30]:
players.corr()

,matches_count,wins_count,easy_wins_count,defeated_count,easy_defeated_count,tied,score
matches_count,1.000000,0.698680,0.540964,0.712808,0.564070,0.059870,-0.013523
wins_count,0.698680,1.000000,0.774689,0.001257,0.007334,-0.007464,0.686482
easy_wins_count,0.540964,0.774689,1.000000,0.000708,0.003388,-0.007292,0.532047
defeated_count,0.712808,0.001257,0.000708,1.000000,0.785387,-0.007914,-0.689668
easy_defeated_count,0.564070,0.007334,0.003388,0.785387,1.000000,-0.008920,-0.536226
tied,0.059870,-0.007464,-0.007292,-0.007914,-0.008920,1.000000,-0.021502
score,-0.013523,0.686482,0.532047,-0.689668,-0.536226,-0.021502,1.000000


In [31]:
# players.to_csv('players.csv', index=False)

In [32]:
players = pd.read_csv('players.csv')

In [33]:
players.describe()

,matches_count,wins_count,easy_wins_count,defeated_count,easy_defeated_count,tied,score
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,19.998900,9.948400,6.040100,9.948400,6.040100,0.102100,0.499494
std,4.459943,3.114435,2.432382,3.177664,2.450733,0.315413,0.114237
min,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.062500
25%,17.000000,8.000000,4.000000,8.000000,4.000000,0.000000,0.423077
50%,20.000000,10.000000,6.000000,10.000000,6.000000,0.000000,0.500000
75%,23.000000,12.000000,8.000000,12.000000,8.000000,0.000000,0.576923
max,37.000000,24.000000,20.000000,25.000000,17.000000,3.000000,1.000000


In [34]:
players.head()

,player_id,matches_count,wins_count,easy_wins_count,defeated_count,easy_defeated_count,tied,score
0,DA1X9864,24.0,9.0,4.0,15.0,9.0,0.0,0.375000
1,SZ0S2637,22.0,14.0,10.0,8.0,6.0,0.0,0.636364
2,QX0H6977,20.0,11.0,7.0,8.0,7.0,1.0,0.566667
3,BK3W3327,14.0,11.0,6.0,3.0,3.0,0.0,0.785714
4,EG5I9716,22.0,12.0,9.0,10.0,7.0,0.0,0.545455


## Buscando dados da API

In [35]:
skip = 0
has_more = True
players_api = pd.DataFrame()
while has_more is True:
    result = requests.get('https://api.desafio.online/v1/players/?skip='+str(skip))
    result = result.json()
    has_more = result['metadata']['has_more']
    df = pd.DataFrame(result['body'])
    concat_df = [players_api, df]
    players_api = pd.concat(concat_df)
    skip = skip + 200

In [36]:
players_api.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 199
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   player_name  10000 non-null  object
 1   email        10000 non-null  object
 2   player_id    10000 non-null  object
 3   country      10000 non-null  object
 4   last_login   10000 non-null  object
dtypes: object(5)
memory usage: 468.8+ KB


In [37]:
players_api['player_id'].value_counts()

KA0N8737    1
VK9T2467    1
TW4F9167    1
PB8C8711    1
DT0M3476    1
           ..
DE3C5292    1
PR5C2919    1
YJ8X3721    1
TE7F1489    1
HZ0M9931    1
Name: player_id, Length: 10000, dtype: int64

## Unindo dados da API com os dados criados dos jogadores

In [38]:
players_merged = pd.merge(players, players_api, on='player_id', how='inner')

In [39]:
players_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   player_id            10000 non-null  object 
 1   matches_count        10000 non-null  float64
 2   wins_count           10000 non-null  float64
 3   easy_wins_count      10000 non-null  float64
 4   defeated_count       10000 non-null  float64
 5   easy_defeated_count  10000 non-null  float64
 6   tied                 10000 non-null  float64
 7   score                10000 non-null  float64
 8   player_name          10000 non-null  object 
 9   email                10000 non-null  object 
 10  country              10000 non-null  object 
 11  last_login           10000 non-null  object 
dtypes: float64(7), object(5)
memory usage: 1015.6+ KB


In [40]:
players_merged.head()

,player_id,matches_count,wins_count,easy_wins_count,defeated_count,easy_defeated_count,tied,score,player_name,email,country,last_login
0,DA1X9864,24.0,9.0,4.0,15.0,9.0,0.0,0.375000,Trisha Kessler,trisha_kessler@gmail.com,Myanmar (Burma),2020-11-12
1,SZ0S2637,22.0,14.0,10.0,8.0,6.0,0.0,0.636364,Tangela Graver,tangela_g@gmail.com,Marshall Islands,2020-12-11
2,QX0H6977,20.0,11.0,7.0,8.0,7.0,1.0,0.566667,Eliseo Ewald,eliseo_ewald@gmail.com,Italy,2020-12-20
3,BK3W3327,14.0,11.0,6.0,3.0,3.0,0.0,0.785714,Bernardo Dieterich,dieterich.b@gmail.com,Cambodia,2020-10-11
4,EG5I9716,22.0,12.0,9.0,10.0,7.0,0.0,0.545455,Maurice Wilcoxson,wilcoxson_m@gmail.com,Marshall Islands,2020-12-11


## Exportando dados finais para aplicação dos modelos

In [41]:
# players_merged.to_csv('players_merged.csv', index=False)